# LangChain: Models, Prompts and Output Parsers

## Outline

 * Direct API calls to OpenAI
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers   
Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those you see here..

In [1]:
### Install required packages
!pip install python-dotenv
!pip install langchain
!pip install langchain_community


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Chat API : llma3.2

Let's start with a direct API calls to LLMA3.2.

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate

# Load environment variables
_ = load_dotenv(find_dotenv())

# Initialize Ollama with llama3.2 model running on Docker
llm = ChatOllama(base_url="http://localhost:11434", model="llama3.2")  # Connect to running container


def get_completion(prompt):
    """Generates a completion for the given prompt using Ollama."""
    response = llm.invoke(prompt)
    return response


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_31376\274072915.py:10: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(base_url="http://localhost:11434", model="llama3.2")  # Connect to running container


In [3]:
print(get_completion("What is 1+1?"))

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

Now to motivate the langchain extractions for models, prompts and parsers. Let's say we get an email from customer in a pirate english language other than english.

## 1. Email Translation for Students

In [ ]:
student_email = """
Yo Prof! I’m totally swamped rn with other stuff. Any chance I can get a few more days for the assignment? Would be a lifesaver!
"""

Let's ask LLM to translate the above to American English in a calm and respectful tone.

In [ ]:
style = """American English \
in a calm and respectful tone
"""

In [ ]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{student_email}```
"""

print(prompt)

In [ ]:
response = get_completion(prompt)

In [ ]:
print(response.content.strip())

Imagine you're a student receiving feedback from your Telugu professor written in Telugu. Translating each piece manually could be tedious. With LangChain, you can automate this process effectively.

## Chat API : LangChain

Let's try how we can do the same using LangChain.

In [ ]:
#!pip install --upgrade langchain

### Model

In [ ]:
from langchain_community.chat_models import ChatOllama


In [ ]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOllama(model="llama3.2", temperature=0.0)
chat

### Prompt template

In [ ]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


In [ ]:
prompt_template.messages[0].prompt

In [ ]:
prompt_template.messages[0].prompt.input_variables

In [ ]:
professor_style = """American English \
in a calm and respectful tone
"""

In [ ]:
professor_feedback = """
మీ ప్రెజెంటేషన్ బాగుంది, కానీ మూలాలు స్పష్టంగా చూపించాలి
"""

In [ ]:
professor_messages = prompt_template.format_messages(
                    style=professor_style,
                    text=professor_feedback)

In [ ]:
print(type(professor_messages))
print(type(professor_messages[0]))

In [ ]:
print(professor_messages[0])

In [ ]:
# Call the LLM to translate to the style of the professor message
professor_translated_message = chat(professor_messages)

In [ ]:
print(professor_translated_message.content)

# Scenario: The Curious Student
Imagine you're teaching about SQL Injection Attacks — instead of a one-way lecture, you present it as a back-and-forth conversation between a Professor and a Student.

In [ ]:
# Example SQL Injection scenario
sql_warning = """\
Your login form is vulnerable to SQL Injection. \
An attacker could enter `' OR 1=1 --` in the username field \
and bypass authentication. \
To prevent this, use prepared statements or parameterized queries.\
"""

In [ ]:
# Professor and Student conversation style
professor_student_style = """\
A conversational tone \
between a knowledgeable professor (PROFESSOR) and a curious student (STUDENT)\
"""

In [ ]:
# Format the message using LangChain
sql_messages = prompt_template.format_messages(
    style=professor_student_style,
    text=sql_warning
)
print(sql_messages[0].content)

In [ ]:
# Simulate a response using chat
student_response = chat(sql_messages)
print(student_response.content)

In [ ]:
from IPython.display import Image, display
display(Image(filename='prompt.png', width=600, height=400))

In [ ]:
## Chain-of-Thought Reasoning using a framework called React

In [ ]:
display(Image(filename='outputparsing.png', width=600, height=400))


## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [ ]:
{
  "workshop_experience": "The workshop on Cloud Computing was fantastic!",
  "key_sessions": "Introduction to Cloud, Hands-on Labs, Career Insights, Q&A",
  "invitation_arrival": "2",
  "price_value": 
    "I found the workshop fees slightly higher than other tech workshops" 
}

In [ ]:
student_feedback = """
The workshop on Cloud Computing was fantastic! It had four key sessions: 
Introduction to Cloud, Hands-on Labs, Career Insights, and Q&A. 
The invitation arrived in just two days, allowing me to plan accordingly. 
I found the workshop fees slightly higher than other tech workshops,
but the content and resources provided made it totally worth it.
"""


In [ ]:

feedback_template = """
For the following text, extract the following information:

workshop_experience: Summarize the student's overall impression of the workshop in one sentence.

key_sessions: List the key workshop sessions mentioned, separated by commas.

invitation_arrival: How many days did it take for the invitation or registration details \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the workshop fee or value.

Format the output as JSON with the following keys:
workshop_experience
key_sessions
invitation_arrival
price_value

text: {text}
"""

In [ ]:
prompt_template = ChatPromptTemplate.from_template(feedback_template)
print(prompt_template)
messages = prompt_template.format_messages(text=student_feedback)


In [ ]:
response = chat(messages)
print(response.content)

### Parse the LLM output string into a Python dictionary

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [ ]:
workshop_experience_schema = ResponseSchema(
    name="workshop_experience",
    description="Summarize the student's overall impression of the workshop in one sentence."
)

key_sessions_schema = ResponseSchema(
    name="key_sessions",
    description="List the key workshop sessions mentioned, separated by commas."
)

invitation_arrival_schema = ResponseSchema(
    name="invitation_arrival",
    description="How many days did it take for the invitation or registration details \
    to arrive? If this information is not found, output -1."
)

price_value_schema = ResponseSchema(
    name="price_value",
    description="Extract any sentences about the workshop fee or value, \
    and output them as a comma-separated Python list."
)

response_schemas = [
    workshop_experience_schema,
    key_sessions_schema,
    invitation_arrival_schema,
    price_value_schema
]

In [ ]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
format_instructions = output_parser.get_format_instructions()

In [ ]:
print(format_instructions)

In [ ]:
feedback_template1 = """
For the following text, extract the following information:

workshop_experience: Summarize the student's overall impression of the workshop in one sentence.

key_sessions: List the key workshop sessions mentioned, separated by commas.

invitation_arrival: How many days did it take for the invitation or registration details \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the workshop fee or value.

text: {text}
{format_instructions}

"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template=feedback_template1)
messages = prompt.format_messages(
    text=student_feedback, 
    format_instructions=format_instructions
)

In [ ]:
print(messages[0].content)

In [ ]:
response = chat(messages)

In [ ]:
print(response.content)

In [ ]:
output_dict = output_parser.parse(response.content)

In [ ]:
output_dict

In [ ]:
output_dict.get('key_sessions')